In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the cleaned data created in 01_ecommerce_eda.ipynb
df = pd.read_csv("../data/processed/online_retail_cleaned.csv", low_memory=False)

# Basic checks
print(df.shape)
print(df.columns.tolist())
df.head()


In [ ]:
# Ensure correct dtypes
df['invoice_date'] = pd.to_datetime(df['invoice_date'], errors='coerce')
df['customer_id']  = df['customer_id'].astype(int, errors='ignore')
df['order_month']  = df['invoice_date'].dt.to_period('M').dt.to_timestamp()

# Sanity: total revenue should match your KPI from Day 1
total_revenue = df['total_price'].sum()
total_orders  = df['invoice_id'].nunique()
total_customers = df['customer_id'].nunique()
print(f"Revenue £{total_revenue:,.2f} | Orders {total_orders:,} | Customers {total_customers:,}")


In [ ]:
# Revenue by customer
cust_rev = (df.groupby('customer_id')['total_price']
              .sum()
              .sort_values(ascending=False))
cust_rev.head(10)
# Repeat vs one-time buyers
orders_per_customer = df.groupby('customer_id')['invoice_id'].nunique()
repeat_rate = (orders_per_customer.gt(1).mean()) * 100
one_time = (orders_per_customer.eq(1).sum())
repeat_customers = (orders_per_customer.gt(1).sum())
print(f"Repeat rate: {repeat_rate:.1f}%  | One-time: {one_time}  | Repeat: {repeat_customers}")
# Top 10 customers (table)
top_customers = cust_rev.head(10).reset_index().rename(columns={'total_price':'revenue'})
top_customers
# Plot: Top 10 customers by revenue
ax = top_customers.set_index('customer_id')['revenue'].plot(kind='bar', figsize=(10,5), title='Top 10 Customers by Revenue')
ax.set_xlabel('Customer ID'); ax.set_ylabel('Revenue (£)')
plt.tight_layout(); plt.show()
# Plot: Top 10 customers by revenue
ax = top_customers.set_index('customer_id')['revenue'].plot(kind='bar', figsize=(10,5), title='Top 10 Customers by Revenue')
ax.set_xlabel('Customer ID'); ax.set_ylabel('Revenue (£)')
plt.tight_layout(); plt.show()


In [ ]:
# Revenue by product
prod_rev = (df.groupby('description')['total_price']
              .sum()
              .sort_values(ascending=False))

top_products = prod_rev.head(10).reset_index().rename(columns={'total_price':'revenue'})
top_products
# Plot: Top 10 products
ax = top_products.set_index('description')['revenue'].plot(kind='bar', figsize=(10,5), title='Top 10 Products by Revenue')
ax.set_xlabel('Product'); ax.set_ylabel('Revenue (£)')
plt.tight_layout(); plt.show()


In [ ]:
country_rev = (df.groupby('country')['total_price']
                 .sum()
                 .sort_values(ascending=False))

country_rev.head(10)
# Exclude UK to see international distribution
intl_rev = country_rev.drop(labels=['United Kingdom'], errors='ignore')
top_intl = intl_rev.head(10).reset_index().rename(columns={'total_price':'revenue'})
ax = top_intl.set_index('country')['revenue'].plot(kind='bar', figsize=(10,5), title='Top 10 Countries (Ex-UK) by Revenue')
ax.set_xlabel('Country'); ax.set_ylabel('Revenue (£)')
plt.tight_layout(); plt.show()


In [ ]:
monthly_sales = df.groupby('order_month')['total_price'].sum().sort_index()
ax = monthly_sales.plot(kind='line', figsize=(10,5), title='Monthly Sales (£)')
ax.set_xlabel('Month'); ax.set_ylabel('Sales (£)')
plt.tight_layout(); plt.show()

monthly_orders = df.groupby('order_month')['invoice_id'].nunique().sort_index()
ax = monthly_orders.plot(kind='line', figsize=(10,5), title='Monthly Orders')
ax.set_xlabel('Month'); ax.set_ylabel('Orders')
plt.tight_layout(); plt.show()


In [ ]:
# Reference date = last date in data + 1 day
ref_date = df['invoice_date'].max() + pd.Timedelta(days=1)

rfm = (df.groupby('customer_id')
         .agg(
             recency=('invoice_date', lambda s: (ref_date - s.max()).days),
             frequency=('invoice_id', 'nunique'),
             monetary=('total_price', 'sum')
         )
      ).reset_index()

# Quick look
rfm.sort_values('monetary', ascending=False).head(10)
# Basic segmentation (very rough thresholds — you can tune)
rfm['segment'] = np.where((rfm['recency']<=30) & (rfm['frequency']>=3) & (rfm['monetary']>=rfm['monetary'].median()),
                          'Champions','Others')
rfm['segment'].value_counts()


In [ ]:
outdir = "../data/processed/"
cust_rev.reset_index().rename(columns={'total_price':'revenue'}).to_csv(outdir+"customer_revenue.csv", index=False)
prod_rev.reset_index().rename(columns={'total_price':'revenue'}).to_csv(outdir+"product_revenue.csv", index=False)
country_rev.reset_index().rename(columns={'total_price':'revenue'}).to_csv(outdir+"country_revenue.csv", index=False)
rfm.to_csv(outdir+"rfm_customers.csv", index=False)
"Saved: customer_revenue.csv, product_revenue.csv, country_revenue.csv, rfm_customers.csv"
